In [127]:
import numpy as np
import pandas as pd # csv files
import random
from math import sqrt 

iris=pd.read_csv('seeds.csv')

In [128]:
 def gauss(x,mean,std):
    # x - jedna z wartości sample [a,b,c,d]
    if x<(mean-(sqrt(6)*std)) or x>(mean+(sqrt(6)*std)):
        return 0
    if (mean-(sqrt(6)*std))<=x<=mean:
        return ((x-mean)/(6*std**2))+(1/(sqrt(6)*std))
    if mean<x<=(mean+(sqrt(6)*std)):
        return (-(x-mean)/(6*std**2))+(1/(sqrt(6)*std))


In [129]:
def naive_bayess_clasifier(training_set, sample_to_classify, iris_species_list):
    data = ProcessingData(training_set)
    stds = data.std()
    means = data.mean()
    probabilities = {}
    for name in iris_species_list:
        probabilities[name]=1
        for key in sample_to_classify.keys():
            probabilities[name]*=(gauss(sample_to_classify[key],means[name][key],stds[name][key]))
    guess = max(probabilities, key= lambda x: probabilities[x])
    return guess

In [130]:
class ProcessingData:
    def __init__(self, flowers_table):
        self.flowers_table = flowers_table

    def stiffle(self):
        x = self.flowers_table.copy()
        for i in range(len(x)-1,0,-1):
            tmp = random.randint(0,i)
            x.iloc[i], x.iloc[tmp] = x.iloc[tmp], x.iloc[i]
        return ProcessingData(x)
    
    def normalize(self):
        flower_data = self.flowers_table.loc[:, self.flowers_table.columns!='Type']
        iris_species = self.flowers_table.loc[:, self.flowers_table.columns=='Type']
        normalized_df=(flower_data-flower_data.min())/(flower_data.max()-flower_data.min())
        normalized_df['Type']=iris_species
        return ProcessingData(normalized_df)    
    
    def mean(self):    
        mean = pd.DataFrame()
        for i, name in enumerate(self.flowers_table.Type.unique()):
            mean[name] = self.flowers_table[self.flowers_table['Type'] == name].mean(axis=0)  
        return mean
    
    def MAX(self):    
        MAX = pd.DataFrame()
        for i, name in enumerate(self.flowers_table.Type.unique()):
            MAX[name] = self.flowers_table[self.flowers_table['Type'] == name].max(0)
        return MAX
    
    def MIN(self):    
        MIN = pd.DataFrame()
        for i, name in enumerate(self.flowers_table.Type.unique()):
            MIN[name] = self.flowers_table[self.flowers_table['Type'] == name].min(0)
        return MIN
    
    def std(self): # można zastąpić p.std()
        self.mean()
        std = self.mean().copy()
        for Type in self.flowers_table['Type'].unique():
            for prop in ['Area','Perimeter',
                                 'Compactness',
                                 'Kernel.Length',
                                 'Kernel.Width',
                                 'Asymmetry.Coeff',
                                 'Kernel.Groove']:
                
                
                std[Type][prop]=self.flowers_table[self.flowers_table['Type']==Type][prop].sub(self.mean()[Type][prop]).pow(2).mean()
        return std
    
    def split(self,a):
        x = self.flowers_table
        n = len(x)
        tabA=x[0:int(a*n)]
        tabB=x[int(a*n):n]
        return tabA, tabB

In [131]:
Iris = ProcessingData(iris)



In [132]:
irisTrain2, irisVal2 = Iris.split(0.7)

irisTrain, irisVal = Iris.stiffle().normalize().split(0.7)


In [133]:
types = [1,2,3]
accuracy=0


In [134]:
for i in irisVal.iloc:
    sample = pd.DataFrame()
    #print(i['sepal.length'].item)
    
    
    
    sample = {
        'Area':i['Area'],
     'Perimeter':i['Perimeter'],
     'Compactness':i['Compactness'],
     'Kernel.Length':i['Kernel.Length'],
     'Kernel.Width':i['Kernel.Width'],
     'Asymmetry.Coeff':i['Asymmetry.Coeff'],
     'Kernel.Groove':i['Kernel.Groove']
    }
    
    answear = naive_bayess_clasifier(irisTrain,sample,types)
    if(answear == i.Type):
        accuracy+=1
print('accuracy = ',(accuracy/len(irisVal)*100),'%')


accuracy =  38.333333333333336 %


In [ ]:
for i in irisVal2.iloc:
    sample = pd.DataFrame()
    #print(i['sepal.length'].item)
    
    
    
    sample = {
        'Area':i['Area'],
     'Perimeter':i['Perimeter'],
     'Compactness':i['Compactness'],
     'Kernel.Length':i['Kernel.Length'],
     'Kernel.Width':i['Kernel.Width'],
     'Asymmetry.Coeff':i['Asymmetry.Coeff'],
     'Kernel.Groove':i['Kernel.Groove']
    }
    
    answear = naive_bayess_clasifier(irisTrain2,sample,types)
    if(answear == i.Type):
        accuracy+=1
print('accuracy = ',(accuracy/len(irisVal)*100),'%')

In [ ]:
class KNN:
    @staticmethod
    def minkowskiMetric(v1,v2,m):

        dim = len(v1)-1
        distance=0
        for i in range(dim):
            distance+=(abs(v1[i]-v2[i])) #**m)
        #distance = distance #**(1/m)
        return distance


    
    @staticmethod
    def KLASTERYZACJA(testSample, x, k, classes):
        # odległość
        distances=[]
        for i in x.index:
            distances.append(KNN.minkowskiMetric(testSample,x.iloc[i],2))
        
        # sortowanie - po distances, ale zamieniamy w treningowym
        
        xs = x.copy()
        
        n=len(distances)
        
        i=0
        while(i<n):
            temp = distances[i]
            tempx=xs.iloc[i]
            ind = distances.index(min(distances[i:n]))
            distances[ind], distances[i] = distances[i], distances[ind]
            xs.iloc[i], xs.iloc[ind] = xs.iloc[ind], xs.iloc[i]
            i+=1

        #while n>1:
        #    for i in range(0,n-1,1):
        #        if distances[i+1]>distances[i]:
        #            distances[i+1], distances[i] = distances[i], distances[i+1]
        #            xs.iloc[i], xs.iloc[i+1] = xs.iloc[i+1], xs.iloc[i]
        #    n-=1
        
        # głosowanie
        for i in range(0,k,1):
            classes[xs.iloc[i].Type]+=1
            
        # zwrot wyniku
        return max(classes,key=classes.get)

In [ ]:
for k in [4]:
    corrected=0
    for i in range(len(irisVal)):
        classes = {1:0, 2:0, 3:0}
        final = KNN.KLASTERYZACJA(irisVal.iloc[i],irisTrain,k,classes)
        if (final == irisVal.iloc[i].Type):
            corrected+=1

    accuracy=corrected/len(irisVal)*100
    print('accuracy for k=',k,'is', accuracy)
    
for k in [4]:
    corrected=0
    for i in range(len(irisVal2)):
        classes = {1:0, 2:0, 3:0}
        final = KNN.KLASTERYZACJA(irisVal2.iloc[i],irisTrain2,k,classes)
        if (final == irisVal2.iloc[i].Type):
            corrected+=1

    accuracy=corrected/len(irisVal2)*100
    print('accuracy for k=',k,'is', accuracy)

In [ ]:
def soft_set(x, Irs):
    scores={1:0,2:0,3:0}
    mx = ProcessingData(Irs).MAX()
    mn = ProcessingData(Irs).MIN()

    for i in [1,2,3]:
        for param in irisTrain.columns.tolist()[:-1]:
            scores[i]+=mn[i][param]<x[param]<mx[i][param]
    return scores


In [ ]:
def soft_main(Irs):
    acc = 0
    for index, row in Irs.iterrows():
        scores = soft_set(row,Irs)
        if max(scores, key=scores.get)==row["Type"]:
            acc+=1
    return acc/len(Irs)

In [ ]:
print(f"accuracy of soft set classifier on normalized data is {soft_main(irisTrain)*100}%")
print(f"accuracy of soft set classifier data \"as is\" is {soft_main(irisTrain2)*100}%")